# Telco Customer Curn Using Artificial Neural Network

`Sri Wahyuni`

# 1 - Introduction

Customer churn adalah kehilangan pelanggan dari suatu bisnis. Churn dihitung dari berapa banyak pelanggan meninggalkan bisnis Anda dalam waktu tertentu. Customer churn penting diketahui bisnis karena merupakan gambaran kesuksesan suatu bisnis dalam mempertahankan pelanggan.

Faktanya, mendapat pelanggan baru 5 kali lebih mahal daripada mempertahankan pelanggan yang sudah ada, dan membuat pelanggan baru menjadi loyal juga 16 kali lebih mahal. Jadi, diperlukan strategi untuk menghentikan customer churn atau kehilangan pelanggan dan meretensi pelanggan yang sudah Anda punya, karena merekalah sumber utama revenue bisnis! [referensi](https://blog.usetada.com/id/apa-itu-customer-churn-dan-bagaimana-menghentikannya).

### Problem Statement and Goal

Customer churn salah satu faktor apakah perusahaan akan berkembang atau tidak sehingga untuk mengurangi customer churn akan dilakukan prediksi apakah konsumen akan mengalami churn atau tidak?

Project ini diharapkan dapat menghasilkan model yang akurat untuk memprediksi churn rate sehingga dapat menjadi bahan evaluasi bagian marketing yang selanjutnya menentukan langkah langkah penangannnya dan diharapkan juga model ini dapat mengetahui faktor faktor apa saja yang mempengaruhi churn rate.

### Output

Customer Churn ini akan berupa klassifikasi `Yes` dan `No`. Analisis akan menggunakan Artificial Neural Network (ANN) dengan menggunakan 2 model yaitu **Sequential** dan **Functional**.

### Dataset : 

Dataset yang digunakan pada project ini adalah dataset yang diperoleh dari `kaggle` dengan nama `Telco Customer Churn` yang terdiri dari 21 kolom dengan 7043 data atau baris. [Klik](https://www.kaggle.com/datasets/blastchar/telco-customer-churn) untuk langsung menuju dataset.

Tabel Informasi : 

| Feature | Description |
|---|---|
|customerID|A unique ID that identifies each customer|
|gender|The customer’s gender: Male, Female|
|SeniorCitizen|Indicates if the customer is 65 or older: Yes, No|
|Partner|Indicates if the customer is Partner: Yes, No|
|Dependents|Indicates if the customer lives with any dependents: Yes, No. Dependents could be children, parents, grandparents, etc.|
|tenure|total amount of months that the customer has been with the company by the end of the quarter specified above.|
|PhoneService| Indicates if the customer subscribes to home phone service with the company: Yes, No|
|MultipleLines|Indicates if the customer subscribes to multiple telephone lines with the company: Yes, No|
|InternetService|Indicates if the customer subscribes to Internet service with the company: No, DSL, Fiber Optic, Cable.|
|OnlineSecurity|Indicates if the customer subscribes to an additional online security service provided by the company: Yes, No|
|OnlineBackup|Indicates if the customer subscribes to an additional online backup service provided by the company: Yes, No|
|DeviceProtection|Indicates if the customer subscribes to an additional device protection plan for their Internet equipment provided by the company: Yes, No|
|TechSupport|Indicates if the customer subscribes to an additional technical support plan from the company with reduced wait times: Yes, No|
|StreamingTV|Indicates if the customer uses their Internet service to stream television programing from a third party provider: Yes, No. The company does not charge an additional fee for this service.|
|StreamingMovies|Indicates if the customer uses their Internet service to stream movies from a third party provider: Yes, No. The company does not charge an additional fee for this service.|
|Contract|Indicates the customer’s current contract type: Month-to-Month, One Year, Two Year.|
|PaperlessBilling|Indicates if the customer has chosen paperless billing: Yes, No|
|PaymentMethod|Indicates how the customer pays their bill: Bank Withdrawal, Credit Card, Mailed Check|
|MonthlyCharges|Indicates the customer’s current total monthly charge for all their services from the company.|
|TotalCharges|Indicates the customer’s total charges, calculated to the end of the quarter specified above.|
|Churn|Whether the customer churned or not (Yes or No)|

The data set includes information about:

- Services – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies
- Customer information – how long they’ve been a customer, contract, payment method, paperless billing, monthly charges, and total charges
- Demographic – gender, age range, and if they have partners and dependents

# 2 - Import Libraries

Pada Project kali ini akan menggunakan library **Pandas**, **Numpy**, **Matplotlib**, **Seaborn**, **Scikit-Learn** dan **Tensorflow**

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
# Import Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, StandardScaler
from sklearn.feature_selection import chi2, SelectKBest
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input, concatenate
from tensorflow.keras import Model
from sklearn.metrics import classification_report 
from tensorflow.keras.models import load_model


# 3 - Data Loading

In [ ]:
# Data Loading

data = pd.read_csv(r'WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [ ]:
# Get all Columns
pd.set_option('display.max_columns', None)

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
# Duplicate Dataset

data_duplicate = data.copy()

In [ ]:
# Check Dataset - 1
data.info()

Berdasarkan data diatas diperoleh beberapa informasi:
- Terdapat **7043 baris**
- Terdapat **21 Kolom**
- Terdapat **1** data bertipe **float**
- Terdapat **2** data bertipe **integer**
- Terdapat **18** data bertipe **object**
- Pada kolom TotalCharges bertipe **object** akan diubah menjadi **float**
- Tidak ada Missing value
- TotalCharges bertipe data **object** akan diubah menjadi float

In [ ]:
# TotalCharges convert to float
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')

In [ ]:
# SeniorCitizen convert to string
data['SeniorCitizen'] = data.SeniorCitizen.map({1 : 'Yes', 0 : 'No'})

In [ ]:
# Remove customerID

data = data.drop(columns='customerID')

In [ ]:
# Check Dataset - 2

data.describe()           

Berdasarkan data diatas diperoleh beberapa informasi:
- Tenur yang paling banyak adalah 72 bulan atau 6 tahun.
- MonthlyCharges sangat bervariasi dimana minimumnya di 18.25 dan paling besar 118.75.
- TotalCharge sepertinya memiliki skew dilihat dari nial mean dan modus yang cukup jauh.

# 4 - Exploratory Data Analysis (EDA)

### 4.1 Churn

In [ ]:
# Number of Churn
data.Churn.value_counts()

In [ ]:
# Get plot and pie for churn

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
fig = sns.countplot(x='Churn', data=data)
for p in fig.patches:
    height = p.get_height()
    fig.text(p.get_x() + p.get_width()/2.,
            height + 3,
            '{:1.0f}'.format(height),
            ha="center")
plt.title('Churn Count')
fig.set_xticklabels(['No', 'Yes'])
plt.xlabel('Churn')
plt.ylabel('Count')
plt.subplot(1, 2, 2)
plt.pie(data['Churn'].value_counts(), labels=['No', 'Yes'], autopct='%1.1f%%', explode=[0.1, 0])
plt.title('Churn Percentage')
plt.show()

Dataset pada kolom 'Churn' memiliki persentase yang cukup jauh dimana No lebih banyak yaitu 74% yang artinya persentase konsumen yang tidak churn jauh lebih banyak dibandingkan consumen yang churn, data ini termasuk data **Imbalance**. 

### 4.2 Gender

In [ ]:
# Get plot and pie for Gender

plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.pie(data['gender'].value_counts(), labels=['Female', 'Male'], autopct='%1.1f%%', explode=[0.1, 0])
plt.title('Gender Percentage')
plt.subplot(1,2,2)
sns.countplot(x='gender', hue='Churn', data=data)
plt.title('Gender vs customer churn')
plt.show()


Jenis kelamin pria dan wanita bisa dikatakan sama karena perbedaan persentasenya hanya 1 persen dan berdasarkan plot jenis kelamin pria dan wanita memiliki data yang hampir sama sehingga saya asumsikan tidak memiliki pengaruh terhdapat customer yang mengalamin churn.

## 4.3 Senior Citizen

In [ ]:
plt.figure(figsize=(20,6))
plt.subplot(1,3,1)
sns.countplot(x='SeniorCitizen', data=data)
plt.title('SeniorCitizen Count')

plt.subplot(1,3,2)
plt.pie(data[data.SeniorCitizen == 'Yes'].Churn.value_counts(), labels=data[data.SeniorCitizen == 'No'].Churn.value_counts().index, autopct='%1.1f%%', explode=[0, 0.1])
plt.title('Churn Percentage by Senior Citizen = Yes')

plt.subplot(1, 3, 3)
plt.pie(data[data.SeniorCitizen == 'No'].Churn.value_counts(), labels=data[data.SeniorCitizen == 'Yes'].Churn.value_counts().index, autopct='%1.1f%%', explode=[0, 0.1])
plt.title('Churn Percentage by Senior Citizen = No')
plt.show()

Rata rata konsumen adalah yang berusia dibawah 65 tahun, Jika berdasarkan persentasi yang mengalami churn paling banyak adalah konsumen yang berusia diatas 65 tahun yaitu 41.7% sedangkan konsumen yang mengalami churn dibawah 65 tahun hanya 23.6%

## 4.4 Tenure

In [ ]:
# Get Function for distribution data

# Histograme
def histogram_plot(df, variable):
    plt.figure(figsize=(12,6))
    plt.subplot(1,2,1)
    sns.histplot(df[variable], kde=True, bins=30)
    plt.axvline(df[variable].mean(), color='r', linestyle='--', label='Mean')
    plt.axvline(df[variable].median(), color='y', linestyle='--', label='Median')
    plt.title('Histogram')

# Boxplot
    plt.subplot(1,2,2)
    sns.boxplot(y=df[variable])
    plt.title('Boxplot')
    plt.show
    print('\nSkewness Value : ', df[variable].skew())
    print('Min Value: ', df[variable].min())
    print('Max Value: ', df[variable].max())

In [ ]:
histogram_plot(data, 'tenure')

Pada kolom tenure berdasarkan nilai skew dan garis mean dam median data termasuk berdistribusi normal dan data tidak memiliki outlier. Namun, dilihat bahwa tenure minimal adalah 0 ini tidak biasa maka akan saya cek data yang 0 tersebut

In [ ]:
print('Number of unique values in tenure: ', data.tenure.nunique())
print('Unique values in tenure: ', data.tenure.unique())

In [ ]:
data['tenure'].value_counts()

In [ ]:
data[data.tenure == 0]

Terdapat 11 data tenure yang nilainya 0 dan jika dilihat pada kolom TotalCharge Nan semua yang artinya tidak ada data. Karena tidak mengetahui kenapa data tersebut demikian maka saya asumsikan bahwa ada kesalan iput dan jumlahnya sedikit sehingga saya akan drop data tersebut di handling missing value.

## 4.5 Payment Method

In [ ]:
plt.figure(figsize=(27,6))
plt.subplot(1,2,1)
sns.countplot(x='PaymentMethod', data=data)
plt.title('PaymentMethod Count')

plt.subplot(1,2,2)
sns.countplot(y='PaymentMethod', hue='Churn', data=data)
plt.title('PaymentMethod vs customer churn')
plt.show()

Berdasarkan data diatas electronic check yang paling banyak digunakan dan memiliki jumlah konsumen churn paling banyak juga sehingga saya asumsikan PaymentMethod memiliki pengaruh terhadap konsumen churn

## 4.6 Internet Service

In [ ]:
plt.figure(figsize=(20,6))
plt.subplot(1,2,1)
sns.countplot(x='InternetService', data=data)
plt.title('InternetService Count')

plt.subplot(1,2,2)
sns.countplot(x='InternetService', hue='Churn', data=data)
plt.title('InternetService vs customer churn')
plt.show()

Terdapat 3 layanan internet service yaitu DSL, Fiber Optic dan No (Tidak menggunakan layanan internet), konsumen paling banyak menggunakan internet service Fiber optic. Namuun Fiber optic juga memiliki konsumen churn yang sangat banyak.

In [ ]:
data[data.InternetService == 'DSL']

In [ ]:
data[data.InternetService == 'Fiber optic']

In [ ]:
data[data.InternetService == 'No']

Dari tabel diatas dapat disimpulkan bahwa jika konsumen tidak menggunakan internet service makan si konsumen tidak akan menggunakan OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies

# 5 - Data Preprocessing

## 5.1 Get Data for Model Inference

In [ ]:
# Get Data for Model Inference

data_inf = data.sample(10, random_state=75)
data_inf

Data Inference sudah diperoleh yaitu sebanyak 10 data, selanjutnya remove data inference dari dataset

In [ ]:
# Remove Data Inference from Dataset

data_train_test = data.drop(data_inf.index)
data_train_test

Data inference sudah berhasil di remove dari dataset, selanjutnya akan mereset index untuk meminimalisir jika ada kesalahan yang dikarenakan oleh index

In [ ]:
# Reset Index

data_train_test.reset_index(drop=True, inplace=True)
data_inf.reset_index(drop=True, inplace=True)


## 5.2 Handling Outlier

### 5.2.1 Outlier Tenure

In [ ]:
# Get Function for distribution data

# Histograme
def histogram_plot(df, variable):
    plt.figure(figsize=(12,6))
    plt.subplot(1,2,1)
    sns.histplot(df[variable], kde=True, bins=30)
    plt.title('Histogram')

# Boxplot
    plt.subplot(1,2,2)
    sns.boxplot(y=df[variable])
    plt.title('Boxplot')
    plt.show

    print('\nSkewness Value : ', df[variable].skew())

In [ ]:
histogram_plot(data_train_test, 'tenure')

Tidak terdapat outlier pada kolom `Tenur`

### 5.2.2 Outlier Monthly Charges

In [ ]:
histogram_plot(data_train_test, 'MonthlyCharges')

Tidak ada Outlier pada kolom `MonthlyCharges`

### 5.2.3 Outlier Total Charges

In [ ]:
histogram_plot(data, 'TotalCharges')

Tidak ada outlier pada kolom `TotalCharges`

## 5.3 Handling Missing Values

In [ ]:
# Check Missing Values on X_train
data_train_test.isnull().sum()

Terdapat missing value pada kolom Totalcharges, sesuai dengan EDA akan drop missing value

In [ ]:
data_train_test = data_train_test.dropna()

In [ ]:
data_train_test.isnull().sum()

## 5.2 Splitting

In [ ]:
# Splitting between X and y

X = data_train_test.drop('Churn', axis=1)
y = data_train_test['Churn']

In [ ]:
# Convert y to 1 and 0
y = y.map({'Yes': 1, 'No':0})

In [ ]:
# Splitting between Train-set, Val-set, Test-set

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=30)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.15, random_state=30)

print('Train Size :', X_train.shape)
print('Val size:', X_val.shape)
print('Test size :', X_test.shape)

## 5.5 Get Numerical Columns and Categorical Columns

In [ ]:
# Get Numerical Columns and Categorical Columns

num_columns = X_train.select_dtypes(include=np.number).columns.tolist()
cat_columns = X_train.select_dtypes(include=['object']).columns.tolist()

print('Numerical Columns: ', num_columns)
print('-----------------------------------------------------')
print('Categorical Columns :', cat_columns)

In [ ]:
# Split Train set, Val set anf Test set 

X_train_num = X_train[num_columns]
X_val_num = X_val[num_columns]
X_test_num = X_test[num_columns]

X_train_cat = X_train[cat_columns]
X_val_cat = X_val[cat_columns]
X_test_cat = X_test[cat_columns]

In [ ]:
# Feature Scaling using MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train_num)

X_train_num_scaled = scaler.transform(X_train_num)
X_val_num_scaled = scaler.transform(X_val_num)
X_test_num_scaled = scaler.transform(X_test_num)


In [ ]:
# Feature Encoding using OneHotEncoder

encoder = OrdinalEncoder()
encoder.fit(X_train_cat)

X_train_cat_encoded = encoder.transform(X_train_cat)
X_val_cat_encoded = encoder.transform(X_val_cat)
X_test_cat_encoded = encoder.transform(X_test_cat)

In [ ]:
X_train_cat_encoded

In [ ]:
# Concat numerical and categorical 

X_train_final = np.concatenate([X_train_num_scaled, X_train_cat_encoded], axis=1)
X_val_final = np.concatenate([X_val_num_scaled, X_val_cat_encoded], axis=1)
X_test_final = np.concatenate([X_test_num_scaled, X_test_cat_encoded], axis=1)

In [ ]:
X_train_final

In [ ]:
# Convert to DataFrame

X_train_final_df = pd.DataFrame(X_train_final, columns=[num_columns+cat_columns])
X_val_final_df = pd.DataFrame(X_val_final, columns=[num_columns + cat_columns])
X_test_final_df = pd.DataFrame(X_test_final, columns=[num_columns + cat_columns])

## 5.6 Feature Selection

In [ ]:
# Feature selected using Chi Square

test = SelectKBest(score_func=chi2, k=6)
fit = test.fit(X_train_final, y_train)

In [ ]:
np.round_(fit.scores_)

In [ ]:
chi = pd.Series(fit.scores_)
chi.index = X_train_final_df.columns
chi.sort_values(ascending=False)

Dari hasil chi2 score feature yang akan digunakan `Contract`, `OnlineSecurity`, `TechSupport`, `OnlineBackup`,`tenure`,`DeviceProtection`, `SeniorCitizen`      

In [ ]:
X_train_final_df = X_train_final_df[['Contract', 'OnlineSecurity', 'TechSupport', 'OnlineBackup','tenure','DeviceProtection', 'SeniorCitizen']]
X_val_final_df = X_val_final_df[['Contract', 'OnlineSecurity', 'TechSupport', 'OnlineBackup','tenure','DeviceProtection', 'SeniorCitizen']]
X_test_final_df = X_test_final_df[['Contract', 'OnlineSecurity', 'TechSupport', 'OnlineBackup','tenure','DeviceProtection', 'SeniorCitizen']]

## 5.7 Pipeline

Membuat pipeline menggunakan `tf.data`

In [ ]:
# Training Pipeline
train_pipe = tf.data.Dataset.from_tensor_slices((X_train_final_df, y_train)).shuffle(500).batch(55)

# Validation Pipeline
val_pipe = tf.data.Dataset.from_tensor_slices((X_val_final_df, y_val)).batch(55)

# Test Pipeline
# Validation Pipeline
test_pipe = tf.data.Dataset.from_tensor_slices((X_test_final_df, y_test)).batch(55)

for row in train_pipe.take(2):
  print(row)

# 6 - Model Definition

## 6.1 Sequential API

In [ ]:
# Model Sequential API

model_sequential = Sequential()
model_sequential.add(Dense(32, activation='relu', input_shape=(X_train_final_df.shape[1],)))
model_sequential.add(Dense(18, activation='relu'))
model_sequential.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile Sequential API

model_sequential.compile(loss='binary_crossentropy',
                         optimizer='Adam',
                         metrics=['accuracy'])

model_sequential.summary()

In [ ]:
# Plot Layers

tf.keras.utils.plot_model(model_sequential, show_shapes=True)

## 6.2 Functional API

In [ ]:
# Model Functional API

input_data = Input(shape=(X_train_final_df.shape[1],))
hidden_1 = Dense(32, activation='relu')(input_data)
hidden_2 = Dense(18, activation='relu')(hidden_1)

concat_data = concatenate([input_data, hidden_2])
output_data = Dense(1, activation='sigmoid')(concat_data)

model_functional = Model(inputs=input_data, outputs=output_data)

In [ ]:
# Compile Functional API

model_functional.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
model_functional.summary()

In [ ]:
# Plot Layers Functional API

tf.keras.utils.plot_model(model_functional, show_shapes=True)

# 7 - Model Training

## 7.1 Sequential API

In [ ]:
# Model Training for Sequential API

%%time
history_sequential = model_sequential.fit(train_pipe, epochs=100, validation_data=val_pipe)

## 7.2 Functional API

In [ ]:
# Model Training for Sequential API

%%time
history_functional = model_functional.fit(train_pipe, epochs=100, validation_data=val_pipe)

# 8 - Model Evaluation

## 8.1 Model Evaluation

### 8.1.1 Sequential API

In [ ]:
# Create DataFrame

history_sequential_df = pd.DataFrame(history_sequential.history)
history_sequential_df

In [ ]:
# Plot between accuracy and val_accuracy

history_sequential_df[['accuracy', 'val_accuracy']].plot()

In [ ]:
# Plot between loss and val_loss

history_sequential_df[['loss', 'val_loss']].plot()

In [ ]:
# Model Evaluation

y_pred_seq = model_sequential.predict(X_test_final_df)
y_pred_seq = np.where(y_pred_seq >=0.5, 1, 0)

print(classification_report(y_test, y_pred_seq))

### 8.1.2 Functional API

In [ ]:
# Create DataFrame

history_functional_df = pd.DataFrame(history_functional.history)
history_functional_df

In [ ]:
# Plot between accuracy and val_accuracy

history_functional_df[['accuracy', 'val_accuracy']].plot()

In [ ]:
# Plot between loss and val_loss

history_functional_df[['loss', 'val_loss']].plot()

In [ ]:
# Model Evaluation Functional API
y_pred_func = model_functional.predict(X_test_final_df)
y_pred_func = np.where(y_pred_func >=0.5, 1, 0)
print(classification_report(y_test, y_pred_func))

## 8.2 Model Improvement

### 8.2.1 Sequential API

In [ ]:
# Model Improvement Sequential API

model_sequential2 = tf.keras.models.Sequential()
model_sequential2.add(tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train_final_df.shape[1],),kernel_initializer='HeNormal',kernel_regularizer='l2'))
model_sequential2.add(tf.keras.layers.Dense(18, activation='relu'))
model_sequential2.add(tf.keras.layers.Dropout(0.2))
model_sequential2.add(tf.keras.layers.Dense(1, activation='sigmoid'))


In [ ]:
# Model Compile
model_sequential2.compile(loss='binary_crossentropy', optimizer='adamax', metrics=[tf.keras.metrics.BinaryAccuracy(threshold=.6)])

In [ ]:
# Training for Sequential API

%time
history_sequential2 = model_sequential2.fit(train_pipe, batch_size= 15, epochs=100, validation_data=val_pipe)

In [ ]:
model_sequential2.summary()

In [ ]:
# Create DataFrame

history_sequential_df2 = pd.DataFrame(history_sequential2.history)
history_sequential_df2

In [ ]:
# Plot between accuracy and val_accuracy

history_sequential_df2[['binary_accuracy', 'val_binary_accuracy']].plot()

In [ ]:
# Plot between loss and val_loss

history_sequential_df2[['loss', 'val_loss']].plot()

### 8.2.2 Model Improvement Functional API

In [ ]:
# Model Improvement Functional API

input_data = Input(shape=(X_train_final_df.shape[1],))
hidden_1 = Dense(32, activation='relu')(input_data)
hidden_2 = Dense(18, activation='relu')(hidden_1)

concat_data = concatenate([input_data, hidden_2])
output_data = Dense(1, activation='sigmoid')(concat_data)

model_functional2 = Model(inputs=input_data, outputs=output_data)

# Compile Functional API

model_functional2.compile(loss='binary_crossentropy', optimizer='Adamax', metrics=[tf.keras.metrics.BinaryAccuracy(threshold=.7)])

In [ ]:
# Training for Functional API

%%time
history_functional2 = model_functional2.fit(train_pipe, epochs=100, validation_data=val_pipe)

In [ ]:
# Create DataFrame

history_functional_df2 = pd.DataFrame(history_functional2.history)
history_functional_df2

In [ ]:
# Plot between accuracy and val_accuracy

history_functional_df2[['binary_accuracy', 'val_binary_accuracy']].plot()

In [ ]:
# Plot between loss and val_loss

history_functional_df2[['loss', 'val_loss']].plot()

In [ ]:
# Model Improvement Sequential API

y_pred_seq2 = model_sequential2.predict(X_test_final_df)
y_pred_seq2 = np.where(y_pred_seq2 >=0.5, 1, 0)

# Model Improvement Functional API
y_pred_func2 = model_functional2.predict(X_test_final_df)
y_pred_func2 = np.where(y_pred_func2 >=0.5, 1, 0)

print('---------------------Sequential API------------------')
print(classification_report(y_test, y_pred_seq2))
print('---------------------Functional API------------------')
print(classification_report(y_test, y_pred_func2))

## 8.2 Model Analysis

- Model Sequential dan funcional API memiliki accurasy sama yaitu 77%
- Recall and F1 score jelek hanya 44 dan 51 hal ini diakibatkan data **imbalance**
- untuk mengatasi data imbalance salah satunya adalah pengaturan threshold. tidak melakukan under sampling karena data yang dimiliki sedikit sehingga akan banyak data yang hilang. tidak melakukan upper sampling karena memiliki kemungkinan terjadinya bias
- untuk improvment menambahkan thresdhold, otimizer `Adamax`, kernel `dropout 0.3`, dihasilkan accuracy meningkat 1% menjadi 78% drngan nilai recaal dan f1 score meningkat.
- Best model adalah sequential setelah improvement

# 9 - Model Saving

In [ ]:
# save 
with open('model_scaler.pkl', 'wb') as file_1:
  joblib.dump(scaler, file_1)

with open('model_encoder.pkl', 'wb') as file_2:
  joblib.dump(encoder, file_2)

with open('num_columns.txt', 'w') as file_3:
  json.dump(num_columns, file_3)

with open('cat_columns.txt', 'w') as file_4:
  json.dump(cat_columns, file_4)

# Freeze Model
model_sequential2.trainable = False

# Save ANN Model
model_sequential2.save('WA_Fn-UseC_-Telco-Customer-Churn.h5')
  

# 10 - Model Inference

In [ ]:
# Load The Models

with open('model_scaler.pkl', 'rb') as file_1:
  model_scaler = joblib.load(file_1)

with open('model_encoder.pkl', 'rb') as file_2:
  model_encoder = joblib.load(file_2)

with open('num_columns.txt', 'r') as file_3:
  num_columns = json.load(file_3)

with open('cat_columns.txt', 'r') as file_4:
  cat_columns = json.load(file_4)

model_ann = load_model('WA_Fn-UseC_-Telco-Customer-Churn.h5')

In [ ]:
# Get Data Inference
data_inf

In [ ]:
# Get numerical and categorical data inference
data_inf_num = data_inf[num_columns]
data_inf_cat = data_inf[cat_columns]

In [ ]:
# scaling and encoding data inference
data_inf_num_scaled = model_scaler.transform(data_inf_num)
data_inf_cat_encoded = model_encoder.transform(data_inf_cat)

# concate data inference
data_inf_final = np.concatenate([data_inf_num_scaled, data_inf_cat_encoded], axis=1)

# convert to DataFrame
data_inf_final_df = pd.DataFrame(data_inf_final, columns=[num_columns+cat_columns])

In [ ]:
# Feature selection data inference
data_inf_final_df2 = data_inf_final_df[['Contract', 'OnlineSecurity', 'TechSupport', 'OnlineBackup','tenure','DeviceProtection', 'SeniorCitizen']]

In [ ]:
# Predict using Neural Network

y_pred_inf = model_ann.predict(data_inf_final_df2)
y_pred_inf = np.where(y_pred_inf >= 0.5, 'Yes', 'No')

In [ ]:
y_pred_inf_df = pd.DataFrame(y_pred_inf, columns=['Prediction'])
y_pred_inf_df

In [ ]:
# Concate between Inference-Set and Prediction

pd.concat([data_inf, y_pred_inf_df], axis=1)

# 11 - Pengambilan Kesimpulan

# Overall Analysis

**Based on EDA** :

- Dataset yang digunakan adalah data yang diperoleh dari `kaggle` dengan nama `Telco Customer Churn` yang terdiri dari 21 kolom dengan 7043 data atau baris.

- Dataset ini merupakan data binary classifikasi untuk menentukan apakah konsumen akan mengalami churn atau tidak churn dengan interpresentasi class 0 (NOT Churn) dan class 1 (Churn).

- Data ini **imbalance** karena persentase `No` yaitu 73,5% berbeda jauh dengan `Yes` yaitu 26,5%.

- Pada data ini lebih banyak konsumen yang berusia dibawah 65 tahun. 

- Konsumen terlama yaitu tenure 72 bulan / 6 tahun sedangkan konsumen terbanyak adalah konsumen dengan tenure 1 bulan, hal dapat diatasi dengan promo paket 3 bulan / 6 bulan / 12 bulan.

- Konsumen yang menggunakan internet service banyak yang mengalami churn hal bisa terjadi karena semakin banyak iklan produk serupa yang lebih menarik diinternet, sehingga harus ditingkat iklan diinternet.

**Based on Model** :

- Model yang digunakan adalah Sequential API dan Functional API, dari kedua model tersebut Best modelnya adalah sequential API setelah dilakukan improvement

**Improvement** : Karena keterbatasan waktu saya melakukan trail dan eror pada epoch 100 dan 200 dan pada epoch 200 hasilnya jelek grafik loss dan val loss tidak beraturan/ tidak konsisten. sehingga bisa mencoba untuk trail eror lebih dalam atau menggunakan earli stopping

### Referensi
- https://community.ibm.com/community/user/businessanalytics/blogs/steven-macko/2019/07/11/telco-customer-churn-1113